In [17]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # 앞으로는 이렇게 mnist를 호출하면 됌.

sess = tf.Session()
%matplotlib inline  
print ("packs loaded")

packs loaded


In [18]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = np.eye(10)[Y_train]
Y_test = np.eye(10)[Y_test]


In [55]:
x = tf.placeholder("float", [None, 784]) 
y = tf.placeholder("float", [None, 10])  # None is for infinite 
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# LOGISTIC REGRESSION MODEL
actv = tf.nn.softmax(tf.matmul(x, W) + b) 
# COST FUNCTION
#cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv), reduction_indices=1)) 
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(actv,1e-10,1.0)), reduction_indices=1))

# OPTIMIZER
learning_rate = 0.1
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [56]:

# PREDICTION
pred = tf.equal(tf.argmax(actv, 1), tf.argmax(y, 1))    
# ACCURACY
accr = tf.reduce_mean(tf.cast(pred, "float"))


In [57]:
# INITIALIZER
init = tf.initialize_all_variables()

In [58]:
init_new_vars_op = tf.initialize_variables([W, b])
sess.run(init_new_vars_op)

In [59]:
training_epochs = 50
batch_size      = 100
display_step    = 5
# SESSION
sess = tf.Session()
sess.run(init)


# MINI-BATCH Learning
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = len(X_train)//batch_size
    
    for i in range(num_batch):
        batch_xs = X_train[batch_size*i:batch_size*(i+1)] 
        batch_xs = batch_xs.reshape(batch_size,-1) # size 바꿔주기
        batch_ys = Y_train[batch_size*i:batch_size*(i+1)] # 

        # 학습
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        feeds = {x: batch_xs, y: batch_ys}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
        
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        
        X_test = X_test.reshape(-1,28*28)
        feeds_test = {x: X_test, y: Y_test}
        
        train_acc = sess.run(accr, feed_dict = feeds_train)
        test_acc = sess.run(accr, feed_dict = feeds_test)
        
        print("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")        
        

Epoch: 000/050 cost: 11.115731800 train_acc: 0.520 test_acc: 0.576
Epoch: 005/050 cost: 7.450523755 train_acc: 0.680 test_acc: 0.676
Epoch: 010/050 cost: 7.281541759 train_acc: 0.720 test_acc: 0.687
Epoch: 015/050 cost: 6.687514750 train_acc: 0.760 test_acc: 0.723
Epoch: 020/050 cost: 6.350994575 train_acc: 0.740 test_acc: 0.713
Epoch: 025/050 cost: 6.398500339 train_acc: 0.760 test_acc: 0.729
Epoch: 030/050 cost: 6.372788136 train_acc: 0.740 test_acc: 0.729
Epoch: 035/050 cost: 6.025481526 train_acc: 0.770 test_acc: 0.742
Epoch: 040/050 cost: 5.938367077 train_acc: 0.760 test_acc: 0.737
Epoch: 045/050 cost: 6.206234492 train_acc: 0.750 test_acc: 0.730
DONE


# weight, bias값이 nan로 채워지는 문제 발생
원인: cross_entropy에서 확률값이 0~1 사이 값이여야 하는데 채워지는 문제 발생 <br>

해결: 
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(actv,1e-10,1.0)), reduction_indices=1)) 

-> 
아마 0~1사이 확률값을 주기 위해 clip을 주는 함수로 이해함
